- **[データ集め](#データ集め)**
- **[学習部分](#学習部分)**
    - [ニューラルネットワーク](#ニューラルネットワーク)
    - [パラメータ](#パラメータ)
    - [データセット](#データセット)
    - [各手法](#各手法)
    - [学習開始](#学習開始)
    - [グラフ作成](#グラフ作成)
    - [精度確認](#精度確認)
- **[モデル保存](#モデル保存)**

# データ集め

In [1]:
from copy import deepcopy

import numpy as np

from osero_learn import learn
from BitBoard import osero

In [2]:
PLAY_WAY = deepcopy(osero.PLAY_WAY)
del PLAY_WAY["human"]
PLAY_WAY = PLAY_WAY.values()

eva = [1 for i in range(64)]

eva_custom = [
     1.0, -0.6,  0.6,  0.4,  0.4,  0.6, -0.6,  1.0,
    -0.6, -0.8,  0.0,  0.0,  0.0,  0.0, -0.8, -0.6,
     0.6,  0.0,  0.8,  0.6,  0.6,  0.8,  0.0,  0.6,
     0.4,  0.0,  0.6,  0.0,  0.0,  0.6,  0.0,  0.4,
     0.4,  0.0,  0.6,  0.0,  0.0,  0.6,  0.0,  0.4,
     0.6,  0.0,  0.8,  0.6,  0.6,  0.8,  0.0,  0.6,
    -0.6, -0.8,  0.0,  0.0,  0.0,  0.0, -0.8, -0.6,
     1.0, -0.6,  0.6,  0.4,  0.4,  0.6, -0.6,  1.0
]

In [3]:
data = []
result = []

turn_vari = [i for i in range(1, 61)]
run = learn(0, 0, check_point=turn_vari, eva=[0, 0])
i = 1

for black in PLAY_WAY:
    print("\r%d/%d" % (i, len(PLAY_WAY)), end="")
    for white in PLAY_WAY:
        run.setup()
        run.black_method = black
        run.white_method = white
        if black == osero.PLAY_WAY["nhand"]:
            run.eva[1] = eva
        elif black == osero.PLAY_WAY["nhand_custom"]:
            run.eva[1] = eva_custom
        if white == osero.PLAY_WAY["nhand"]:
            run.eva[0] = eva
        elif white == osero.PLAY_WAY["nhand_custom"]:
            run.eva[0] = eva_custom
        data_ele , result_ele = run.play()
        for data_each_turn in data_ele:
            data.append(data_each_turn)
        for result_each_turn in result_ele:
            result.append(result_each_turn)
    i += 1

data = np.array(data).astype(np.float32)
result = np.array(result).astype(np.float32)

5/5

# 学習部分

In [4]:
from chainer.datasets import TupleDataset
from chainer.datasets import split_dataset_random
from chainer.iterators import SerialIterator

import chainer
import chainer.links as L
import chainer.functions as F

from chainer import optimizers
from chainer.optimizer_hooks import GradientHardClipping

from chainer.dataset import concat_examples

## ニューラルネットワーク

In [5]:
class Net(chainer.Chain):
    def __init__(self):
        n_in = 192
        n_hidden = 192
        n_out = 1
        super().__init__()
        with self.init_scope():
            self.l1 = L.Linear(n_in, n_hidden)
            self.l2 = L.Linear(n_hidden, n_hidden)
            self.l3 = L.Linear(n_hidden, n_hidden)
            self.l4 = L.Linear(n_hidden, n_hidden)
            self.l5 = L.Linear(n_hidden, n_hidden)
            self.l6 = L.Linear(n_hidden, n_hidden)
            self.l7 = L.Linear(n_hidden, n_hidden)
            self.l8 = L.Linear(n_hidden, n_hidden)
            self.l9 = L.Linear(n_hidden, n_hidden)
            self.l10 = L.Linear(n_hidden, n_out)
        
    def __call__(self, x):
        h = 2 * F.tanh(self.l1(x))
        h = 2 * F.tanh(self.l2(h))
        h = 2 * F.tanh(self.l3(h))
        h = 2 * F.tanh(self.l4(h))
        h = 2 * F.tanh(self.l5(h))
        h = 2 * F.tanh(self.l6(h))
        h = 2 * F.tanh(self.l7(h))
        h = 2 * F.tanh(self.l8(h))
        h = 2 * F.tanh(self.l9(h))
        h = self.l10(h)

        return h

## パラメータ

In [22]:
batch_size = 200
bound = 2
n_epoch = 100

## データセット

In [23]:
dataset = TupleDataset(data, result)

train_val, test = split_dataset_random(dataset, int(len(dataset) * 0.7), seed=0)
train, valid = split_dataset_random(train_val, int(len(train_val) * 0.7), seed=0)

train_iter = SerialIterator(train, batch_size=batch_size, repeat=True, shuffle=True)

## 各手法

In [24]:
net = Net()

optimizer = optimizers.AdaBound()
optimizer.setup(net)

for param in net.params():
    if param.name != "b":
        param.update_rule.add_hook(GradientHardClipping(-bound, bound))

## 学習開始

In [25]:
results_train = {
    "MSE": [],
    "MAE": []
}
results_valid = {
    "MSE": [],
    "MAE": []
}

train_iter.reset()

for epoch in range(n_epoch):
    print("\r%d/%d" % (epoch + 1, n_epoch), end="")
    while True:
        train_batch = train_iter.next()

        x_train, t_train = concat_examples(train_batch)

        y_train = net(x_train)
        MSE_train = F.mean_squared_error(y_train, t_train)
        MAE_train = F.mean_absolute_error(y_train, t_train)

        net.cleargrads()
        MSE_train.backward()

        optimizer.update()

        if train_iter.is_new_epoch:
            with chainer.using_config("train", False), chainer.using_config("enable_backprop", False):
                x_valid, t_valid = concat_examples(valid)
                y_valid = net(x_valid)
                MSE_valid = F.mean_squared_error(y_valid, t_valid)
                MAE_valid = F.mean_absolute_error(y_valid, t_valid)

            results_train["MSE"].append(MSE_train.array)
            results_train["MAE"].append(MAE_train.array)
            results_valid["MSE"].append(MSE_valid.array)
            results_valid["MAE"].append(MAE_valid.array)

            break

100/100

## グラフ作成

In [26]:
import matplotlib.pyplot as plt

def two_plot(y, xlabel, ylabel, title, save_dir):
    fig = plt.figure(figsize=(10, 10))
    plt.plot(y[0], label="train")
    plt.plot(y[1], label="valid")
    plt.minorticks_on()
    plt.grid(which="major")
    plt.grid(which="minor", linestyle="--")
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend()
    plt.savefig(save_dir + "/" + title)
    plt.clf()
    plt.close()

In [27]:
two_plot(
    [results_train["MSE"], results_valid["MSE"]],
    "epoch",
    "mean squared error",
    "mean squared error each epoch",
    "fig/model_test"
)

two_plot(
    [results_train["MAE"], results_valid["MAE"]],
    "epoch",
    "mean absolute error",
    "mean absolute error each epoch",
    "fig/model_test"
)

## 精度確認

In [28]:
x_test, t_test = concat_examples(test)

with chainer.using_config("train", False), chainer.using_config("enable_backprop", False):
    y_test = net(x_test)
    MSE_test = F.mean_squared_error(y_test, t_test)
    MAE_test = F.mean_absolute_error(y_test, t_test)

print("test MSE: {:.4f}".format(MSE_test.array))
print("test MAE: {:.4f}".format(MAE_test.array))

test MSE: 220.4348
test MAE: 10.8649


# モデル保存

In [29]:
# chainer.serializers.save_npz("deep_AI.net", net)